In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 4. ACRIMA models evaluation

The researchers working on the ACRIMA data prediction model also included their models along with the dataset. Therefore, we'll try to use their models on the Kaggle dataset and see how they're performing compared to the simple CNN models in the previous notebook.

Each model is saved in a JSON file, with the algorithm weights being saved in a h5 file (Hierarchical Data Format). The files can be found [here](https://drive.google.com/drive/folders/133uwNfPLYQs2fyfmAfpWQXH4Zggkb3aF?usp=sharing).

In [21]:
import os
import numpy as np
from utils import get_testing_data, load_acrima_model, draw_conf_matrix

ImportError: ignored

In [3]:
# Load images for evaluation
kaggle_dir = "drive/MyDrive/kaggle_dataset"
kaggle_g_dir = os.path.join(kaggle_dir, "glaucoma")
kaggle_n_dir = os.path.join(kaggle_dir, "normal")

X_g, y_g = get_testing_data([kaggle_g_dir], (299, 299))
X_n, y_n = get_testing_data([kaggle_n_dir], (299, 299))

In [4]:
models_dir = "drive/MyDrive/pre-trained-models"

#### 4.1. InceptionV3

In [5]:
inception_model = load_acrima_model("InceptionV3", models_dir)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 149, 149, 32  96         ['conv2d_1[0][0]']               
 rmalization)                   )                                                           

We notice that the model already has a final dense layer for classification, so no more layers need to be added.

In [17]:
y_g_pred = inception_model.predict(X_g, batch_size=20)
y_n_pred = inception_model.predict(X_n, batch_size=20)

# Change data type from float to int
y_g_pred = y_g_pred.astype(int)
y_n_pred = y_n_pred.astype(int)

# Concatenate the dataframes for the accuracy analysis
y_test = np.hstack([y_g, y_n])
print(y_test.shape)
y_test_pred = np.vstack([y_g_pred, y_n_pred])
print(y_test_pred.shape)

(401,)
(401, 2)


We also notice that the ACRIMA models have a double-digit prediction rather than a single-digit one, where:
* (1, 0) means that the patient has glaucoma
- (0, 0) means that the patient doesn't have glaucoma
Therefore, we'll convert the values in the `y_test_pred` numpy array to single digit labels. 

In [19]:
def convert_predictions(predictions_array):
  new_predictions = []
  for row in predictions_array:
    if row[0] == 1 and row[1] == 0:
      new_predictions.append(0)
    elif row[0] == 0 and row[1] == 0:
      new_predictions.append(1)
  
  new_predictions = np.array(new_predictions)
  return new_predictions

In [20]:
y_test_pred = convert_predictions(y_test_pred)
y_test_pred.shape

(401,)

In [ ]:
draw_conf_matrix(y_test, y_test_pred)

#### 4.2. ResNet50

In [7]:
resnet_model = load_acrima_model("ResNet50", models_dir)

TypeError: ignored

In [ ]:
y_g_pred = resnet_model.predict(X_g, batch_size=20)
y_n_pred = resnet_model.predict(X_n, batch_size=20)

#### 4.3. VGG16

In [ ]:
vgg16_model = load_acrima_model("VGG16", models_dir)

In [ ]:
y_g_pred = resnet_model.predict(X_g, batch_size=20)
y_n_pred = resnet_model.predict(X_n, batch_size=20)

#### 4.4. VGG19

In [ ]:
vgg19_model = load_acrima_model("VGG19", models_dir)

In [ ]:
y_g_pred = resnet_model.predict(X_g, batch_size=20)
y_n_pred = resnet_model.predict(X_n, batch_size=20)

#### 4.5. Xception

In [ ]:
xception_model = load_acrima_model("Xception", models_dir)

In [ ]:
y_g_pred = resnet_model.predict(X_g, batch_size=20)
y_n_pred = resnet_model.predict(X_n, batch_size=20)